## Парсер для VK

### getGroupPostsCount

Возвращает количество постов в группе

__group_name__: Название группы(из юрл)


### getGroupCommentsCount

Возвращает количество комментариев к посту

__post_id__: номер поста

__owner_id__: номер группы


### getGroupPosts
Возвращает список постов

__group_name__: Имя группы

__offset__ : Отступ

__count__: Количество постов

### getPostComments

Возвращает список комментариев к посту


__post_id__: Номер поста

__owner_id__: Номер группы

__offset__ : Отступ

__count__: Количество постов


In [1]:
import time
import vk_api


In [2]:
class vkParser:
    def __init__(self,phone='', password=''):
        self.vk_session = vk_api.VkApi(phone, password)
        self.vk_session.auth()
        self.vk = self.vk_session.get_api()
    
    def getGroupPostsCount(self,group_name):
        posts=self.vk.wall.get(domain=group_name)
        return posts['count']
    
    def getGroupCommentsCount(self,post_id, owner_id):
        comments= self.vk.wall.getComments(owner_id=owner_id,post_id=post_id)
        return comments['count']
        
    
        
    def getGroupPosts(self,group_name,offset=0, count=100):
        #выбираем 100 комментов с нужным отступом. 
        #например 100 с 100ого. 100-200
        posts=self.vk.wall.get(domain=group_name,count=count,offset=offset)
        post_list=[];
        for post in posts['items']:
#             print(post)
            #генерируем данные для записи в таблицу
            #post_id -- id поста
            #from_id -- откуда пост
            #owner_id -- автор
            #date -- дата
            #update_flag (0) -- обновляем отметку для считывания комментов
            #text -- удаляем переносы
            post_list.append({'post_id':post['id'],
                              'from_id':post['from_id'],
                              'owner_id':post['owner_id'],
                              'date':post['date'],
                              'text':post['text'].replace('\n','.').replace("'",''),
                              'likes':post['likes']['count'] if ('likes' in post) else 0 ,
                              'reposts':post['reposts']['count'] if ('reposts' in post) else 0,
                              'views':post['views']['count'] if ('views' in post) else 0,
                              'marked_as_ads':post['marked_as_ads']
                             })
            
        return post_list
    
    def getPostComments(self,post_id, owner_id,offset=50,count=50):
        comments= self.vk.wall.getComments(owner_id=owner_id,post_id=post_id,count=count,offset=offset)
        comment_list=[]
        for comment in comments['items']:
            if(comment.get("deleted") is None):
                comment_list.append({
                    'comment_id':comment['id'], #id коммента
                    'from_id':comment['from_id'], #id автора, те юзер вк
                    'post_id':comment['post_id'], #id поста
                    'owner_id':comment['owner_id'], #id группы
                    'date':comment['date'], #дата
                    'thread_counts':comment['thread']['count'] if ('thread' in comment) else 0, #количество ответов на комментарий
                    'text':comment['text'].replace('\n','.').replace("'",'') #текст сообщения
                });
        return comment_list
        